import libraries

In [1]:
import os
import json
import numpy as np
import pandas as pd
from utils import load_json, load_pickle

Settings

In [2]:
baseline_folder = "../FP_prediction/baseline_models/models_cached/"
MIST_folder = "../FP_prediction/mist/models_cached/"

Consolidate results

In [3]:
# Manually add in (hack)

all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for folder in [MIST_folder, baseline_folder]:

    for model_type in os.listdir(folder):
        subfolder = os.path.join(folder, model_type)
        for dataset in os.listdir(subfolder):
            subsubfolder = os.path.join(subfolder, dataset)
            for checkpoint in os.listdir(subsubfolder):
                if "wo_aux" in checkpoint: continue
                all_folders.append(os.path.join(subsubfolder, checkpoint))

for f in all_folders:

    try:
        expt_name = f.split("/")[-1]
        test_performance = load_json(os.path.join(f, "test_performance.json"))
        for k, v in test_performance.items():
            if k == "threshold": continue 
            all_test_performances[k][expt_name] = round(v, 5)
    except Exception as e:
        print(e)
        continue

Format the table

In [4]:
results = pd.DataFrame.from_dict(all_test_performances)
results = results.reset_index()
results = results.rename(columns = {"index": "expt_name"})
results.loc[:, "dataset"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[0]) 
results.loc[:, "meta"] = results.loc[:, "expt_name"].apply(lambda x: "meta" in x) 
results.loc[:, "model"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[1]) 
results.loc[:, "split"] = results.loc[:, "expt_name"].apply(lambda x: "_".join(x.split("_")[-2:]).replace("4096_", "")) 

results = results.loc[:, ["dataset", "model", "split", "meta", "loss", "jaccard"]]

results = results.sort_values(by = 'split', ascending = False)
results = results.sort_values(by = 'model', ascending = False)
results = results.sort_values(by = 'dataset', ascending = False)

Seperate results into w and wo meta

In [5]:
results_wo_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is False), :]
results_w_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is True), :]

In [6]:
for dataset in ["NIST2023", "MSG", "C"]:
    for split in ["random", "scaffold_vanilla", "inchikey_vanilla"]:

        sub_results_wo_meta = results_wo_meta.loc[results_wo_meta.apply(lambda x : x["dataset"] == dataset and x["split"] == split, axis = 1), :]
        print(sub_results_wo_meta)
        print("*" * 80)
        print() 

    dataset model   split   meta     loss  jaccard
1  NIST2023  MIST  random  False  0.01628  0.62105
********************************************************************************

     dataset    model             split   meta     loss  jaccard
0   NIST2023     MIST  scaffold_vanilla  False  0.06124  0.25526
56  NIST2023   binned  scaffold_vanilla  False  0.05609  0.22640
52  NIST2023       MS  scaffold_vanilla  False  0.05603  0.21677
57  NIST2023  formula  scaffold_vanilla  False  0.04986  0.28083
********************************************************************************

     dataset    model             split   meta     loss  jaccard
55  NIST2023       MS  inchikey_vanilla  False  0.04887  0.23924
54  NIST2023   binned  inchikey_vanilla  False  0.05056  0.23660
53  NIST2023  formula  inchikey_vanilla  False  0.04205  0.30891
2   NIST2023     MIST  inchikey_vanilla  False  0.04789  0.30135
********************************************************************************

  

In [8]:
for dataset in ["NIST2023", "MSG", "C"]:
    for split in ["random", "scaffold_vanilla", "inchikey_vanilla"]:

        sub_results_w_meta = results_w_meta.loc[results_w_meta.apply(lambda x : x["dataset"] == dataset and x["split"] == split, axis = 1), :]
        print(sub_results_w_meta)
        print("*" * 80)
        print() 

     dataset    model   split  meta     loss  jaccard
30  NIST2023  formula  random  True  0.04026  0.32598
17  NIST2023     MIST  random  True  0.01628  0.62107
********************************************************************************

     dataset    model             split  meta     loss  jaccard
27  NIST2023   binned  scaffold_vanilla  True  0.05208  0.23001
32  NIST2023       MS  scaffold_vanilla  True  0.05469  0.22971
29  NIST2023  formula  scaffold_vanilla  True  0.05046  0.28236
15  NIST2023     MIST  scaffold_vanilla  True  0.05966  0.25595
********************************************************************************

     dataset    model             split  meta     loss  jaccard
31  NIST2023   binned  inchikey_vanilla  True  0.04979  0.24493
28  NIST2023       MS  inchikey_vanilla  True  0.04861  0.24342
16  NIST2023     MIST  inchikey_vanilla  True  0.04787  0.30027
33  NIST2023  formula  inchikey_vanilla  True  0.05099  0.22827
**********************************